<a href="https://colab.research.google.com/github/jafetimbre/mib-fiesta/blob/master/src/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proiect la materia "*Stiinta datelor folosind Python*"

Studenti masteranzi:
*   Muraru Madalina-Maria
*   Beresescu Mihai-Cristian
*   Imbre Jozsef-Jafet 


Acest proiect contine atat pregatirea, filtrarea si aranjarea datelor cat si analiza statistica a acestora.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

## 1. Descrierea si preprocesarea datelor

Pentru acest proiect s-au ales doua seturi de date cu o legatura intre acestea. Ambele contin entitati de proprietati aflate in jurul orasului Seattle, Washington, din Statele Unite ale Americii.

*   **Primul set de date**: contine inregistrari despre casele de vanzare in apropierea orasului
*   **Al doilea set de date**: contine inregistrari despre diferite unitati non rezidentiale (scoli, spitale, parcuri, etc.)



### 1.1 Descrierea seturilor de date

Atributele primului set de date (casele de vanzare) :

*   **price**: pretul cu care a fost vanduta locuinta
*   **bedrooms**:  numarul dormitoarelor
*   **bathrooms**: numarul de bai/dormitoare
*   **sqft_living**: suprafata utila locuibila
*   **sqft_lot**: ntreaga suprafata
*   **floors**: numarul etajelor
*   **waterfront**: locuinta pe malul unei ape
*   **condition**: gradul de confort per total
*   **grade**: nota locuintel dupa standardul Seattle
*   **yr_built**: anul construirii
*   **yr_renovated**: anul renovarii
*   **zipcode**: codul postal
*   **lat**: latitudinea
*   **long**: longitudinea

De sigur, aceste campuri se filtreaza si se efectueaza transformarile aferente (ex. sqrft -> m2)

In [2]:
housing_columns = [ 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
                    'condition', 'grade', 'sqft_basement', 'yr_built', 'yr_renovated' ,'zipcode', 'lat', 'long' ]

housing_data = pd.read_csv('https://raw.githubusercontent.com/jafetimbre/mib-fiesta/master/data/housing.csv', usecols=housing_columns)
housing_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,grade,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,20141013T000000,221900,3,1.00,1180,5650,1.0,0,3,7,0,1955,0,98178,47.5112,-122.257
1,20141209T000000,538000,3,2.25,2570,7242,2.0,0,3,7,400,1951,1991,98125,47.7210,-122.319
2,20150225T000000,180000,2,1.00,770,10000,1.0,0,3,6,0,1933,0,98028,47.7379,-122.233
3,20141209T000000,604000,4,3.00,1960,5000,1.0,0,5,7,910,1965,0,98136,47.5208,-122.393
4,20150218T000000,510000,3,2.00,1680,8080,1.0,0,3,8,0,1987,0,98074,47.6168,-122.045


Al doilea set de date ales este un set ajutator, care contine anumite cladiri relevante din imprejurimile caselor primului dataset.

*   BuildingTypeName: Tipul Cladirii
*   PropertyTypeEPA: Propietatea (spital,restaurant,gradinita)
*   City: Orasul
*   Zip: Cod postal
*   Neighborhood: Cartierul
*   Longitude: Longitudinea
*   Latitude: Latitudinea

Asemanator cu primul set, se vor adauga doar campurile necesare. De exemplu, fiind vorba de orasul Seattle, coloana "City" se poate elimina. 


In [3]:
property_columns = [ 'BuildingTypeName', 'PropertyTypeEPA', 'City', 'Zip', 'Neighborhood', 'Latitude', 'Longitude' ]
property_data = pd.read_csv('https://raw.githubusercontent.com/jafetimbre/mib-fiesta/master/data/property_types.csv', usecols=property_columns)
property_data.head()

,BuildingTypeName,PropertyTypeEPA,City,Zip,Neighborhood,Latitude,Longitude
0,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.612190,-122.337997
1,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613106,-122.333358
2,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.613349,-122.337699
3,NonResidential,Hotel,SEATTLE,98101,DOWNTOWN,47.614216,-122.336609
4,NonResidential,Hotel,SEATTLE,98121,DOWNTOWN,47.613754,-122.340924


De sigur, unele coloane au ramas incluse deoarece se vor folosi pentru a manipula/creea alte coloane, dupa care se vor elimina.

### 1.2 Preprocesarea si filtrarea datelor

Ca prim pas, se vor extrage codurile postale comune printr-o simpla intersectie. Aceasta informatie va ajuta la eliminarea inregistrarilor ne-relevante.

In [4]:
housing_data_ZIPs = housing_data.zipcode.unique()
property_data_ZIPs = property_data.Zip.unique()

zip_comon = list(set(housing_data_ZIPs).intersection(property_data_ZIPs))
print(zip_comon)

housing_data = housing_data[housing_data.zipcode.map(lambda e: e in zip_comon)]
property_data = property_data[property_data.Zip.map(lambda e: e in zip_comon)]

[98177, 98178, 98199, 98102, 98103, 98105, 98106, 98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118, 98119, 98122, 98125, 98126, 98133, 98136, 98144, 98146, 98155]


Pentru primul set, se vor transpune unitatile de masura imperiale in unitati de masura metrice, prin formula:

> 1 ft<sup>2</sup> = 0.092 m<sup>2</sup>

Totodata se va transpune formatul de timp corespunzator.

In [5]:
housing_data.sqft_living = housing_data.sqft_living.apply(lambda e: round(e * 0.092))
housing_data.sqft_lot = housing_data.sqft_lot.apply(lambda e: round(e * 0.092))
housing_data.sqft_basement = housing_data.sqft_basement.apply(lambda e: round(e * 0.092))

housing_data.date = housing_data.date.apply(lambda e: datetime.datetime.strptime(e, '%Y%m%dT%H%M%S'))

Deoarece ne intereseaza doar entitatile non-rezidentiale din setul de date secundar, acestea se vor elimina complet, impreuna cu colanele specifice.

In [6]:
property_data = property_data[property_data.BuildingTypeName.map(lambda e: e in ['NonResidential', 'Nonresidential COS', 'SPS-District K-12', 'Campus'])]
property_data = property_data[property_data.City.map(lambda e: e == 'SEATTLE')]

property_data = property_data[property_data.PropertyTypeEPA.map(lambda e: e != 'Hotel')]
property_data = property_data.drop(['BuildingTypeName', 'City'], axis=1)

Pentru o consistenta in numele coloanelor, acestea se redenumesc.

In [7]:
housing_data = housing_data.rename(columns={'sqft_living': 'm2_living', 'sqft_lot':'m2_lot', 'sqft_basement':'m2_basement', 'zipcode':'zip'})
property_data = property_data.rename(columns={'PropertyTypeEPA': 'property_type', 'Zip': 'zip', 'Neighborhood':'neighborhood', 'Latitude': 'latitude', 'Longitude':'longitude'})

Inaintea exportari seturilor de date curatate si pregatite de analiza, se mai elimina valorile nule si se reseteaza indecsi de tabel.

In [8]:
housing_data.dropna(inplace=True)
housing_data.reset_index(drop=True)

property_data.dropna(inplace=True)
property_data.reset_index(drop=True)

housing_data.to_csv('housing_data_cleaned.csv', index = False, header=True)
property_data.to_csv('property_data_cleaned.csv', index = False, header=True)

Dupa curatarea datelor din acesta, primul set arata astfel:

In [9]:
housing_data.head()

,date,price,bedrooms,bathrooms,m2_living,m2_lot,floors,waterfront,condition,grade,m2_basement,yr_built,yr_renovated,zip,lat,long
0,2014-10-13,221900,3,1.00,109,520,1.0,0,3,7,0,1955,0,98178,47.5112,-122.257
1,2014-12-09,538000,3,2.25,236,666,2.0,0,3,7,37,1951,1991,98125,47.7210,-122.319
3,2014-12-09,604000,4,3.00,180,460,1.0,0,5,7,84,1965,0,98136,47.5208,-122.393
8,2015-04-15,229500,3,1.00,164,687,1.0,0,3,7,67,1960,0,98146,47.5123,-122.337
11,2014-05-27,468000,2,1.00,107,552,1.0,0,4,7,28,1942,0,98115,47.6900,-122.292


Iar cel de al doilea, arata astfel:


In [10]:
property_data.head()

,property_type,zip,neighborhood,latitude,longitude
22,K-12 School,98118,SOUTHEAST,47.545701,-122.268584
28,K-12 School,98112,SHARED: CENTRAL & EAST,47.623094,-122.305389
30,K-12 School,98125,NORTH,47.710115,-122.293792
31,K-12 School,98109,MAGNOLIA / QUEEN ANNE,47.633032,-122.352457
32,Office,98103,LAKE UNION,47.663916,-122.331722


## 2. Generarea campurilor de interes

## 3. Formularea ipotezelor

## 4. Analiza datelor

## 5. Concluzii, respectiv validarea/invalidarea ipotezelor